In [1]:
import logging
import time
import random
from datetime import datetime

name = 'CAFI'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('batimento')
logger.setLevel(logging.DEBUG)

# fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_batimento.log', mode='w')
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# fHandler.setFormatter(formatter)
# logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {name}')
# logger.debug('debug message')
# logger.warning('warn message')

In [2]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {name}')
levantamento = pd.read_excel('batimento/lev_fisico/2024_levantamento_ajustes3.xlsx', sheet_name=1)
levantamento = levantamento.rename(columns={"Código":"codigo", "Quantidade":"qtd_lev"})
# display(levantamento.info())

levantamento = levantamento[["codigo", "ED", "Nome", "qtd_lev", "CADMAT", "valor unitário"]]
levantamento = levantamento.drop_duplicates(keep="last")
duplicada = levantamento.duplicated(subset=['codigo'])
# display(len(duplicada))
display(levantamento.info())

suap = pd.read_excel('batimento/suap/2024-01-08_15-39_CAFI_relatorio_almox.xlsx')
suap = suap.rename(columns={"qtd":"qtd_suap"})
display(suap.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 412 entries, 0 to 411
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   codigo          412 non-null    object 
 1   ED              403 non-null    float64
 2   Nome            412 non-null    object 
 3   qtd_lev         412 non-null    int64  
 4   CADMAT          396 non-null    float64
 5   valor unitário  0 non-null      float64
dtypes: float64(3), int64(1), object(2)
memory usage: 22.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   codigo     411 non-null    object 
 1   descricao  411 non-null    object 
 2   und        411 non-null    object 
 3   qtd_suap   411 non-null    int64  
 4   vlr_total  411 non-null    float64
 5   vlr_unt    411 non-null    float64
 6   ed         411 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 22.6+ KB


None

In [3]:
batimento = pd.merge(suap, levantamento, on='codigo', how='outer', indicator=True)
display(batimento.info())
duplicada = batimento[batimento.duplicated(subset=['codigo'])]
display(duplicada)

batimento = batimento.drop_duplicates(subset=['codigo'], keep='last')
batimento.index = [x for x in range(0, len(batimento))]

display(batimento.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 412 entries, 0 to 411
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   codigo          412 non-null    object  
 1   descricao       412 non-null    object  
 2   und             412 non-null    object  
 3   qtd_suap        412 non-null    int64   
 4   vlr_total       412 non-null    float64 
 5   vlr_unt         412 non-null    float64 
 6   ed              412 non-null    object  
 7   ED              403 non-null    float64 
 8   Nome            412 non-null    object  
 9   qtd_lev         412 non-null    int64   
 10  CADMAT          396 non-null    float64 
 11  valor unitário  0 non-null      float64 
 12  _merge          412 non-null    category
dtypes: category(1), float64(5), int64(2), object(5)
memory usage: 42.4+ KB


None

,codigo,descricao,und,qtd_suap,vlr_total,vlr_unt,ed,ED,Nome,qtd_lev,CADMAT,valor unitário,_merge
367,30.26.480,"CABO ELÉTRICO FLEXIVEL 1,5MM VERMELHO TENSÃO 7...",ROLO,3,1.23,0.41,26 - MATERIAL ELETRICO E ELETRONICO,26.0,"CABO ELÉTRICO FLEXIVEL 1,5MM VERMELHO TENSÃO 7...",3,1781014.0,NaN,both


<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 0 to 410
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   codigo          411 non-null    object  
 1   descricao       411 non-null    object  
 2   und             411 non-null    object  
 3   qtd_suap        411 non-null    int64   
 4   vlr_total       411 non-null    float64 
 5   vlr_unt         411 non-null    float64 
 6   ed              411 non-null    object  
 7   ED              402 non-null    float64 
 8   Nome            411 non-null    object  
 9   qtd_lev         411 non-null    int64   
 10  CADMAT          395 non-null    float64 
 11  valor unitário  0 non-null      float64 
 12  _merge          411 non-null    category
dtypes: category(1), float64(5), int64(2), object(5)
memory usage: 42.3+ KB


None

In [4]:
# batimento.to_excel(f'batimento/merge/{data_e_hora}_{name}_batimento.xlsx', index=False)

display(batimento['codigo'])

0      30.14.117
1      30.14.121
2      30.14.122
3      30.14.123
4       30.14.13
         ...    
406    30.42.969
407    30.42.970
408    30.42.982
409    30.42.995
410     30.44.67
Name: codigo, Length: 411, dtype: object

In [5]:
ed = batimento['codigo'].str[3:5]
batimento['elemento'] = ed

In [6]:
batimento['diferenca'] = batimento['qtd_suap'] - batimento['qtd_lev']
info = []

for valor in batimento['diferenca']:
    if valor < 0:
        info.append("baixa no levantamento")
        
    elif valor > 0:
        info.append("baixa no suap")
        
    else: info.append("iguais")

batimento['info'] = info

display(batimento)

,codigo,descricao,und,qtd_suap,vlr_total,vlr_unt,ed,ED,Nome,qtd_lev,CADMAT,valor unitário,_merge,elemento,diferenca,info
0,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",UN,11,162.39,14.76,14 - MATERIAL EDUCATIVO E ESPORTIVO,14.0,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",11,819808.0,NaN,both,14,0,iguais
1,30.14.121,REDE ESPORTE BASQUETE,UN,7,111.93,15.99,14 - MATERIAL EDUCATIVO E ESPORTIVO,14.0,REDE ESPORTE BASQUETE,7,1977482.0,NaN,both,14,0,iguais
2,30.14.122,REDE ESPORTE FUTSAL,UN,10,1099.40,109.94,14 - MATERIAL EDUCATIVO E ESPORTIVO,14.0,REDE ESPORTE FUTSAL,10,1977520.0,NaN,both,14,0,iguais
3,30.14.123,REDE ESPORTE FUTEBOL DE CAMPO,UN,3,261.00,87.00,14 - MATERIAL EDUCATIVO E ESPORTIVO,14.0,REDE ESPORTE FUTEBOL DE CAMPO,3,1738720.0,NaN,both,14,0,iguais
4,30.14.13,BOLA DE FUTEBOL DE CAMPO,UN,5,275.00,55.00,14 - MATERIAL EDUCATIVO E ESPORTIVO,14.0,BOLA DE FUTEBOL DE CAMPO,5,1079778.0,NaN,both,14,0,iguais
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,30.42.969,DISJUNTOR TERMOMAGNETICO TRIPOLAR 125-A MARCA JNG,UN,1,89.99,89.99,42 - FERRAMENTAS,42.0,DISJUNTOR TERMOMAGNETICO TRIPOLAR 125-A MARCA JNG,1,179116.0,NaN,both,42,0,iguais
407,30.42.970,"INTERRUPTOR MANUAL, interruptor de 01 seção, s...",UN,19,100.51,5.29,42 - FERRAMENTAS,42.0,"INTERRUPTOR MANUAL, interruptor de 01 seção, s...",19,1741780.0,NaN,both,42,0,iguais
408,30.42.982,"BROCA DE VIDEA, MATERIAL AÇO RÁPIDO, 10 MM, CO...",UN,5,34.55,6.91,42 - FERRAMENTAS,42.0,"BROCA DE VIDEA, MATERIAL AÇO RÁPIDO, 10 MM, CO...",5,239682.0,NaN,both,42,0,iguais
409,30.42.995,"BROCA DE WÍDIA, 6 MM.",UN,4,14.08,3.52,42 - FERRAMENTAS,42.0,"BROCA DE WÍDIA, 6 MM.",4,239704.0,NaN,both,42,0,iguais


In [7]:
batimento.to_excel(f'batimento/merge/{data_e_hora}_{name}_batimento.xlsx', index=False)